## Step1：Import necessary packages

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Step2：Import dataset

In [17]:
df = pd.read_csv('kidney_disease.csv')
df.head(10)

id   age     bp     sg   al   su       rbc        pc         pcc  \
0   0  48.0   80.0  1.020  1.0  0.0       NaN    normal  notpresent   
1   1   7.0   50.0  1.020  4.0  0.0       NaN    normal  notpresent   
2   2  62.0   80.0  1.010  2.0  3.0    normal    normal  notpresent   
3   3  48.0   70.0  1.005  4.0  0.0    normal  abnormal     present   
4   4  51.0   80.0  1.010  2.0  0.0    normal    normal  notpresent   
5   5  60.0   90.0  1.015  3.0  0.0       NaN       NaN  notpresent   
6   6  68.0   70.0  1.010  0.0  0.0       NaN    normal  notpresent   
7   7  24.0    NaN  1.015  2.0  4.0    normal  abnormal  notpresent   
8   8  52.0  100.0  1.015  3.0  0.0    normal  abnormal     present   
9   9  53.0   90.0  1.020  2.0  0.0  abnormal  abnormal     present   

           ba  ...  pcv     wc   rc  htn   dm  cad appet   pe  ane  \
0  notpresent  ...   44   7800  5.2  yes  yes   no  good   no   no   
1  notpresent  ...   38   6000  NaN   no   no   no  good   no   no   
2  notpresent  ...   31   7500  NaN   no  yes   no  poor   no  yes   
3  notpresent  ...   32   6700  3.9  yes   no   no  poor  yes  yes   
4  notpresent  ...   35   7300  4.6   no   no   no  good   no   no   
5  notpresent  ...   39   7800  4.4  yes  yes   no  good  yes   no   
6  notpresent  ...   36    NaN  NaN   no   no   no  good   no   no   
7  notpresent  ...   44   6900    5   no  yes   no  good  yes   no   
8  notpresent  ...   33   9600  4.0  yes  yes   no  good   no  yes   
9  notpresent  ...   29  12100  3.7  yes  yes   no  poor   no  yes   

  classification  
0            ckd  
1            ckd  
2            ckd  
3            ckd  
4            ckd  
5            ckd  
6            ckd  
7            ckd  
8            ckd  
9            ckd  

[10 rows x 26 columns]

In [18]:
df = df.drop(columns=['id','sg','al','su','pc','pcc','ba','sod','pot','hemo','pcv','htn','dm','cad','pe'])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
age               391 non-null float64
bp                388 non-null float64
rbc               248 non-null object
bgr               356 non-null float64
bu                381 non-null float64
sc                383 non-null float64
wc                295 non-null object
rc                270 non-null object
appet             399 non-null object
ane               399 non-null object
classification    400 non-null object
dtypes: float64(5), object(6)
memory usage: 34.5+ KB


In [20]:
df['wc'] = pd.to_numeric(df['wc'], errors='coerce')
df['rc'] = pd.to_numeric(df['rc'], errors='coerce')

In [21]:
df['classification'] = df['classification'].replace(to_replace = {'ckd\t': 'ckd', 'notckd': 'not ckd'})
df['classification'] = df['classification'].map({'ckd': 0, 'not ckd': 1})
df['classification'] = pd.to_numeric(df['classification'], errors='coerce')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
age               391 non-null float64
bp                388 non-null float64
rbc               248 non-null object
bgr               356 non-null float64
bu                381 non-null float64
sc                383 non-null float64
wc                294 non-null float64
rc                269 non-null float64
appet             399 non-null object
ane               399 non-null object
classification    400 non-null int64
dtypes: float64(7), int64(1), object(3)
memory usage: 34.5+ KB


## Step3: Data preprocessing

In [23]:
df.isna().sum()

age                 9
bp                 12
rbc               152
bgr                44
bu                 19
sc                 17
wc                106
rc                131
appet               1
ane                 1
classification      0
dtype: int64

### Step3.1: Using random sample to fill these null value

In [24]:
for col in df.columns:
    random_sample = df[col].dropna().sample(df[col].isna().sum())
    random_sample.index = df[df[col].isnull()].index
    df.loc[df[col].isnull(), col] = random_sample

In [25]:
df.isna().sum()

age               0
bp                0
rbc               0
bgr               0
bu                0
sc                0
wc                0
rc                0
appet             0
ane               0
classification    0
dtype: int64

### Step3.2: Feature Encoding

In [26]:
for col in [col for col in df.columns if df[col].dtype == 'object']:
    print(f"{col} has {df[col].nunique()} categories\n")

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in [col for col in df.columns if df[col].dtype == 'object']:
    df[col] = le.fit_transform(df[col])

rbc has 2 categories

appet has 2 categories

ane has 2 categories



In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
age               400 non-null float64
bp                400 non-null float64
rbc               400 non-null int64
bgr               400 non-null float64
bu                400 non-null float64
sc                400 non-null float64
wc                400 non-null float64
rc                400 non-null float64
appet             400 non-null int64
ane               400 non-null int64
classification    400 non-null int64
dtypes: float64(7), int64(4)
memory usage: 34.5 KB


## Step4: Build Model

In [28]:
X = df.drop(columns=['classification'])  # Drop the 'target' column to get the features
Y = df['classification'] 

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
classifier = RandomForestClassifier()
classifier.fit(X_train, Y_train)

# accuracy score, confusion matrix and classification report of random forest

accuracy = accuracy_score(Y_test, classifier.predict(X_test))

print(f"Training Accuracy of Random Forest Classifier is {accuracy_score(Y_train, classifier.predict(X_train))}")
print(f"Test Accuracy of Random Forest Classifier is {accuracy}")


Training Accuracy of Random Forest Classifier is 0.9933333333333333
Test Accuracy of Random Forest Classifier is 0.95


/home/lance/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
